##1. Double shifted QR finding all eigenvalues

In [ ]:
import numpy as np
from numpy import linalg
from scipy.linalg import hessenberg
import math
from scipy.io import mmread

# only works for real matrix
def Hessenberg(matrix):
    n = matrix.shape[0]
    m = matrix.shape[1]
    # h, q = hessenberg(matrix, calc_q=True)
    H = matrix.astype(dtype=np.float64)
    for k in range(0, m-2):
        Q = np.identity(n)
        v = H[k+1:, k].copy()
        if np.sign(v[0]) == 0:
            v[0] = v[0] + np.linalg.norm(v)
        else:
            v[0] = v[0] + np.sign(v[0]) * np.linalg.norm(v)
        if np.linalg.norm(v) == 0:
            continue
        v = v/np.linalg.norm(v)
        v = v.reshape([v.shape[0], 1])
        Q[k+1:, k+1:] = Q[k+1:, k+1:] - 2*v*v.T
        # zeroed_column = Q[k + 1:, k + 1:].dot(H[k+1:, k])
        H = Q @ H @ Q.T
    # test = np.allclose(H, h)
    return H

def Francis_double_shift(matrix):
    h = Hessenberg(matrix)
    row_number = matrix.shape[0] - 1
    eigen_values = []
    n = row_number
    tolerance = 1.0e-10
    while n >= 2:
        while abs(h[n, n-1]) > tolerance and abs(h[n-1, n-2]) > tolerance:
            for i in range(0, n-1):
                Q = np.identity(3)
                if i == 0:
                    trace = h[n - 1, n - 1] + h[n, n]
                    det = h[n - 1, n - 1] * h[n, n] - h[n, n - 1] * h[n - 1, n]
                    m11 = h[0, 0] * h[0, 0] + h[0, 1] * h[1, 0] - trace * h[0, 0] + det  # the first 3 non-zero elements
                    m21 = h[1, 0] * (h[0, 0] + h[1, 1] - trace)                          # of the first column of M
                    m31 = h[1, 0] * h[2, 1]
                    v = np.array([[m11],
                                  [m21],
                                  [m31]])
                else:
                    v = h[i: i+3, i-1].copy()
                v[0] = v[0] + np.sign(v[0]) * np.linalg.norm(v)
                if np.linalg.norm(v) == 0:
                    continue
                v = v / np.linalg.norm(v)
                v = v.reshape([v.shape[0], 1])
                Q = Q - 2 * v * v.T                 # Householder reflector
                # test = Q.dot(h[i: i+3, i-1])
                p = np.identity(n + 1)
                p[i:i+3, i:i+3] = Q
                h = p.T @ h @ p
            r = np.linalg.norm(h[n-1:, n-2])
            c = h[n-1, n-2]/r
            s = -h[n, n-2]/r
            G = np.array([[c, -s],                  # Given Rotation
                          [s, c]])
            # test = G.dot(h[n-1:, n-2])
            p = np.identity(n + 1)
            p[i+1:i + 3, i+1:i + 3] = G.T
            h = p.T @ h @ p

        if abs(h[n, n - 1]) < tolerance:
            close_to_zero = lambda x: 0 if(abs(x) < tolerance) else x  # if the number is close to 0,transfer it into 0
            eigen_values.append(close_to_zero(h[n, n]))       # when h[n,n-1] is converging,h[n,n] is the right eigenvalue
            h[n, n - 1] = 0
            h = h[:n, :n]                      # set the active matrix in next iteration
            n = n - 1                          # deflation
        elif abs(h[n - 1, n - 2]) < tolerance:  # when h[n-1,n-2] is converging,the conjugate pair eigenvalues of
            eigen_values.append(np.linalg.eigvals(h[n-1:, n-1:])) # the 2x2 matrix at the bottom right are the right eigenvalues
            h[n - 1, n - 2] = 0
            h = h[:n-1, :n-1]                  # set the active matrix in next iteration
            n = n - 2                          # deflation
    eigen_values.append(np.linalg.eigvals(h))
    return eigen_values

## Covergency 
a. when $h_{n,n-1}$ is converging(close to zero), $h_{n,n}$ is the real number eigenvalue;

b. when $h_{n-1,n-2}$ is converging, the conjugate pair eigenvalue of the 2x2 matrix at the bottom right are the complex number eigenvalues.

##2. Shifted inverse iteration finding the corresponding eigenvector

In [ ]:
def create_eig(eig):
  '''create a eigenvalue list'''
  eigen_list = []
  for num in eig:
    if type(num) is np.ndarray:
      for item in num:
        eigen_list.append(item)
    else:
      eigen_list.append(num)
  return eigen_list



def shifted_inverse_iter(A, eig, num_simulations=50):
    '''
    input: a matrix, eigenvalues out from Francis_double_shift
    return: eigen pair list 
    '''
    eigenvectors=[]
    x0 = np.random.rand(A.shape[1]) # initialize a random vector 
    eigen_list = create_eig(eig)
    for t0 in eigen_list:  
      sigma = t0 - 0.001  # avoid singluar 
      for _ in range(num_simulations): 
        try:
            x1 = np.linalg.solve(A - sigma * np.eye(A.shape[0]), x0) 
        except np.linalg.linalg.LinAlgError:
            print('Error!')
            break
        x0 = x1 / np.linalg.norm(x1)
      eigenvectors.append(x0)
    return eigen_list, eigenvectors


##3. Error calculation
the correct result should satisfy:
V*λ-A*V is close to 0

In [ ]:
def err_cal(A, eig_list, eig_vec):
  err=[]
  i=0
  for i in range(0, len(eig_list)):
      tvector=np.ndarray.__mul__(eig_vec[i].transpose(),eig_list[i])-A@eig_vec[i].transpose() 
      err.append(np.linalg.norm(tvector)) 
      i=i+1
  return err


##4. Test matrices collections

In [ ]:
# has complex conjugate pairs 
A = np.array([[1, 0, 2, 3],
            [-1, 0, 5, 2],
            [2, -2, 0, 0],
            [2, -1, 2, 0]])

# has complex conjugate pairs 
B = np.array([[7, 3, 4, -11, -9, -2],
            [-6, 4, -5, 7, 1, 12],
            [-1, -9, 2, 2, 9, 1],
            [-8, 0, -1, 5, 0, 8],
            [-4, 3, -5, 7, 2, 10],
            [6, 1, 4, -11, -7, -1]])


C = np.array([[5, 2, 0],
            [2, 5, 0],
            [-3, 4, 6]])


# matrix from matrix market
D = mmread('fidapm05.mtx') #real unsymmetric, 27 x 27
D = D.toarray()

E = mmread('bcspwr01.mtx') #pattern symmetric indefinite, 39 x 39
E = E.toarray()

F = mmread('jgl011.mtx') #pattern unsymmetric, 11 x 11
F = F.toarray()


[Matrics D information](https://math.nist.gov/MatrixMarket/data/SPARSKIT/fidap/fidap005.html)

[Matrics E information](https://math.nist.gov/MatrixMarket/data/Harwell-Boeing/bcspwr/bcspwr01.html)

[Matrics F information](https://math.nist.gov/MatrixMarket/data/Harwell-Boeing/counterx/jgl011.html)


##5. Test outcomes

In [ ]:
# Test on a small matrix A

eig = Francis_double_shift(A)
eig_list, eig_vec = shifted_inverse_iter(A,eig)
print('eigenvalues are:\n', eig_list, '\n eigenvectors are:\n', eig_vec)

eigenvalues are:
 [-1.3843814667028846, (-0.5198081181052909+2.930927606958384j), (-0.5198081181052909-2.930927606958384j), 3.42399770291347] 
 eigenvectors are:
 [array([ 0.54238761,  0.64022219,  0.1413405 , -0.52531332]), array([ 0.21043622+0.28808159j, -0.04006835+0.82413515j,
       -0.3840283 -0.10283028j, -0.13203746+0.12820171j]), array([ 0.24814522-0.25631691j,  0.07394031-0.82178893j,
       -0.39453831+0.0489011j , -0.11310093-0.14518178j]), array([-0.24042725+0.72680722j, -0.11912818+0.36012233j,
       -0.07085231+0.21418524j, -0.14703016+0.44446952j])]


In [ ]:
err_cal(A, eig_list, eig_vec)

[2.5311336062892422e-11,
 2.5786599344126235e-11,
 2.578652266316025e-11,
 2.0071159176712822e-11]

In [ ]:
# Test on a small matrix B 

eig_0 = Francis_double_shift(B)
eig_list_0, eig_vec_0 = shifted_inverse_iter(B,eig_0)
print('eigenvalues are:\n', eig_list_0, '\n eigenvectors are:\n', eig_vec_0)

eigenvalues are:
 [2.9999999999997464, 4.000000000001846, (0.9999999999992633+1.999999999999828j), (0.9999999999992633-1.999999999999828j), (4.999999999999936+6.000000000000175j), (4.999999999999936-6.000000000000175j)] 
 eigenvectors are:
 [array([ 0.24701609, -0.12350805,  0.82338697,  0.41169348, -0.12350805,
        0.24701609]), array([ 0.06126789,  0.53915739, -0.45338235, -0.45338235,  0.53915739,
        0.06126789]), array([ 0.05504652-0.15985286j, -0.52464571+0.13061883j,
       -0.47381609+0.2910759j ,  0.15811793+0.09339768j,
       -0.55386084-0.03101657j, -0.10658888-0.13063773j]), array([-0.13404133+0.10303387j, -0.04304533-0.5389448j ,
        0.12524611-0.54179358j,  0.13884134+0.1201977j ,
       -0.20557871-0.51522932j, -0.15775681-0.05949952j]), array([ 8.27564156e-02-4.93103818e-01j, -4.93103818e-01-8.27564156e-02j,
       -5.39996776e-17-1.30449628e-16j,  5.49298345e-17+1.99089007e-17j,
       -4.93103818e-01-8.27564156e-02j,  8.27564156e-02-4.93103818e-01j]), arr

In [ ]:
err_cal(B, eig_list_0, eig_vec_0)

[2.4884408330517533e-13,
 1.8415294664619344e-12,
 7.568464497132229e-13,
 7.560922457363745e-13,
 1.8617034716639197e-13,
 1.852098725755089e-13]

In [ ]:
# Test on matrix D from matrix market 

eig_1 = Francis_double_shift(D)
eig_list_1, eig_vec_1 = shifted_inverse_iter(D,eig_1)
print('eigenvalues are:\n', eig_list_1, '\n eigenvectors are:\n', eig_vec_1)

eigenvalues are:
 [0, 0.5280921768144473, 0.2508690322721493, 2.239548174937626, 2.310346330944573, 2.1361209289936762, 3.9202397356870184, 0.8373627146126169, 4.042048052044995, 4.13437196816613, 3.963623239669481, 1.0608158675486186, 0.9843970286060981, 1.170049702467917, 1.2811887716729682, 1.4222469006754803, 1.4561709895113923, 0.06503580148893845, 1.5141861007945163, 1.534710917700405, 2.0309545740570742, 1.9791880975151073, 6.621812979275903, -0.026919141974032904, -0.03292494443729897, -0.01999161301061853, -0.03968583116715484, -9.460282118466293e-05, -9.720969065160688e-05, -0.00011623889563699052, 6.337401229184664, 6.222292769194472, -0.00010488488359350208, -0.00012741128537462185, -2.621748769386268e-05, -4.577305011001556e-05, -1.1514624140216112e-05, -2.7860446183888546e-06, -0.05078343984252265, 1.5758210106837596, 7.202124453481757, 6.949912061234648] 
 eigenvectors are:
 [array([ 1.01728569e-03,  1.29129061e-03,  1.80464421e-03,  1.82257482e-03,
        2.04496115e-0

In [ ]:
err_cal(D, eig_list_1, eig_vec_1)

[0.00012716299898088413,
 2.030273221515079e-14,
 3.380429467715289e-15,
 7.817290929914578e-14,
 6.411026427149199e-14,
 4.9996620624795064e-14,
 2.329743681319188e-13,
 4.796178112127357e-15,
 2.770100078219611e-13,
 6.443759881977056e-13,
 1.5457044284861764e-13,
 1.2229844318295217e-13,
 9.30181037414979e-14,
 1.3787884657533866e-13,
 1.2113616207991706e-13,
 6.155913525691722e-13,
 5.249908233470861e-13,
 5.770619683281524e-14,
 2.315927948975135e-13,
 1.1735927449742942e-12,
 5.56219166132314e-13,
 1.2648367998292618e-12,
 4.4875052082694455e-12,
 1.6782927994394454e-14,
 4.150284469130803e-15,
 5.10274653324537e-15,
 2.5972157349038473e-14,
 2.5118866019850557e-05,
 2.79791621177097e-05,
 1.0808776116718584e-05,
 5.514005279264599e-12,
 5.499166768011989e-12,
 2.05555913399178e-05,
 3.3533823256277e-06,
 0.0001011716651185623,
 8.163506744212782e-05,
 0.00011589591179938557,
 0.00012462503976257887,
 3.5488793603371663e-13,
 1.7123153263379522e-12,
 2.299324156284878e-14,
 2.695

In [ ]:
# Test on matrix E from matrix market 

eig_2 = Francis_double_shift(E)
eig_list_2, eig_vec_2 = shifted_inverse_iter(E,eig_2)
print('eigenvalues are:\n', eig_2, '\n eigenvectors are:\n', eig_vec_2)

eigenvalues are:
 [1.0000000000000002, 0.9999999999999991, 0.9999999999999999, array([1., 2.]), array([-1.        ,  2.00575035]), 1.6163295948754728, array([0.42993683, 1.58456193]), array([0.63780118, 1.51159324]), array([0.35015607, 0.77351363]), array([1.24409201, 0.22566619]), 0.18305816674045616, array([0.13222035, 1.78201538]), array([ 2.25453664, -0.07307345]), array([-0.37869805,  2.35555428]), -0.4009027714306859, array([-0.54611136,  2.51162345]), array([ 2.77421824, -0.7071975 ]), array([ 2.89635451, -0.9032942 ]), array([ 3.1606575 , -1.16745561]), array([-1.37599792,  3.36397489]), array([ 3.53385145, -1.53220927]), array([-1.63953182,  3.56064285]), array([3.83636324])] 
 eigenvectors are:
 [array([ 5.30712358e-02, -5.55162558e-17, -2.71168707e-17,  4.26325641e-17,
       -1.19976727e-01,  2.83939539e-17,  1.73047963e-01, -3.78169529e-17,
       -5.30712358e-02,  0.00000000e+00, -1.58187385e-02, -2.13093432e-17,
        1.58187385e-02,  1.42037635e-17, -1.58187385e-02, -

In [ ]:
err_cal(E, eig_list_2, eig_vec_2)

[2.4858589755404625e-16,
 9.29170810368278e-16,
 1.3360334298411993e-16,
 1.4353152255760758e-15,
 5.115558261746089e-16,
 6.024203200023295e-16,
 1.4454807814129047e-15,
 2.3610711197046754e-15,
 6.714002118817901e-16,
 2.9886389923795013e-15,
 1.5554550668019474e-15,
 5.614242988227213e-16,
 3.273957564663723e-16,
 9.862421686056267e-16,
 5.121956962021865e-16,
 8.041927373282855e-16,
 4.514624225416591e-16,
 4.598165708474472e-16,
 3.988850278354115e-15,
 1.2952382852918024e-15,
 6.732474553751128e-16,
 3.049229896993935e-16,
 5.035971268738107e-15,
 2.4396929190897045e-16,
 2.9880696217901413e-15,
 2.1232058675428616e-15,
 1.0169322469063802e-14,
 7.00383727752842e-16,
 2.089286609825648e-15,
 4.35948296676807e-16,
 1.2535330776516347e-14,
 5.525502746501075e-16,
 4.81792028556592e-15,
 7.929887598006184e-15,
 8.434971381229691e-16,
 1.4313939130351954e-15,
 2.9584253072721678e-15,
 9.939102659319015e-16,
 6.437723127493397e-15]

In [ ]:
# Test on matrix G from matrix market
# notice G is a singular matrix

eig_3 = Francis_double_shift(F) 
eig_list_3, eig_vec_3 = shifted_inverse_iter(F,eig_3) 
print('eigenvalues are:\n', eig_list_3, '\n eigenvectors are:\n', eig_vec_3)

eigenvalues are:
 [0, 0, 0, 0, 0, 0, (1.2911922699753966+1.7935969040702786j), (1.2911922699753966-1.7935969040702786j), -0.3623776933300358, 1.0000000000000042, 6.779993153379235] 
 eigenvectors are:
 [array([-0.198414  , -0.43396668,  0.11325814,  0.4994374 ,  0.05865578,
       -0.23738465,  0.37714286, -0.198414  ,  0.198414  ,  0.23721078,
       -0.41593965]), array([-0.198414  , -0.43396668,  0.11325814,  0.4994374 ,  0.05865578,
       -0.23738465,  0.37714286, -0.198414  ,  0.198414  ,  0.23721078,
       -0.41593965]), array([-0.198414  , -0.43396668,  0.11325814,  0.4994374 ,  0.05865578,
       -0.23738465,  0.37714286, -0.198414  ,  0.198414  ,  0.23721078,
       -0.41593965]), array([-0.198414  , -0.43396668,  0.11325814,  0.4994374 ,  0.05865578,
       -0.23738465,  0.37714286, -0.198414  ,  0.198414  ,  0.23721078,
       -0.41593965]), array([-0.198414  , -0.43396668,  0.11325814,  0.4994374 ,  0.05865578,
       -0.23738465,  0.37714286, -0.198414  ,  0.198414  ,  0

In [ ]:
err_cal(F, eig_list_3, eig_vec_3)

[2.435541875787129e-16,
 1.4686870114880517e-16,
 2.5888677460363984e-16,
 1.8619006149354548e-16,
 2.0014830212433605e-16,
 2.830524433501838e-16,
 2.4327330568945574e-15,
 2.45946351250256e-15,
 5.112109705309496e-16,
 4.239533572889116e-15,
 2.3814537615548095e-15]